In [1]:
from __future__ import print_function
import os
import sys
from test_arg_utils import *

sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], "src"))

from pkg.utils.test_scripts import *
from pkg.planning.pddlstream.plan_rnb import *

rtype = "panda"
dat_root = "stowing-deep"
res_root = "stowing-deep-result"
dat_dir = "20210917-113211"
file_option = "obj_3c_obs7"
data_idx = 5
cname = "Full"

GRASP_SAMPLE = 30
STABLE_SAMPLE = 50

IK_TIMEOUT_SINGLE = 0.01
IK_TRY_NUM = 8
TIMEOUT_MOTION = 2
MAX_TIME = 100

MAX_ITER = 100
MAX_SKELETONS = 30
SEARCH_SAMPLE_RATIO = 1000

USE_MOVEIT_IK = True
TIMED_COMPLETE = False

VERBOSE = True
VISUALIZE = True
PLAY_RESULT = True
SHOW_STATE = False
USE_PYBULLET_GUI = False
SAVE_RESULTS = False
STACK_TIMELOG= True


CLEARANCE = 1e-3
TOOL_NAME="grip0"
ROBOT_TYPE = {e.name: e for e in RobotType}[rtype]

In [2]:

########################################################
################### Create data folders ################
DATASET_PATH = create_data_dirs(dat_root, rtype, dat_dir)
RESULTSET_PATH = create_data_dirs(res_root, rtype, dat_dir)
print("-"*50)
print("DATASET_PATH: {}".format(DATASET_PATH))
print("RESULTSET_PATH: {}".format(RESULTSET_PATH))
print("-"*50)


########################################################
########## Load scene and prepare planner  #############
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene
crob.home_pose = HOME_POSE
crob.home_dict = list2dict(HOME_POSE, gscene.joint_names)

fname = "data_%s_%02d.pkl" % (file_option, data_idx)
print(fname)
set_meta_data("dat_root", dat_root)
set_meta_data("rtype", rtype)
set_meta_data("dat_dir", dat_dir)
set_meta_data("fname", fname)


file_gtems = os.path.join(DATASET_PATH, fname)
initial_state = load_saved_scene(pscene, file_gtems, VISUALIZE=VISUALIZE)

gscene.NAME_DICT['obj_0'].color = (1,0,0,1)
gscene.update_markers_all()

mplan = MoveitPlanner(pscene)
checkers = get_checkers_by_case_name(cname, pscene)

mplan.motion_filters = checkers


--------------------------------------------------
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep/panda/20210917-113211
RESULTSET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep-result/panda/20210917-113211
--------------------------------------------------
connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
data_obj_3c_obs7_05.pkl


## Edit scene

In [3]:
obj0 = gscene.NAME_DICT['obj_0']
obj1 = gscene.NAME_DICT['obj_1']
obj2 = gscene.NAME_DICT['obj_2']

pscene.set_object_state(initial_state)
gscene.update_markers_all()

# wp = gscene.NAME_DICT["wp"]
# wp_ceil = gscene.copy_from(wp, new_name="wp_ceil")
# # wp_ceil.set_offset_tf(center = np.add(wp.center, (0,0,0.3)))

# # obj0.set_offset_tf(center=[0.29480234, 0.59536386, 0.1798489])
# # obj1.set_offset_tf(center=(0.30, 0.43, obj0.center[2]))
# # obj2.set_offset_tf(center=(0.17, 0.5, obj0.center[2]))

gscene.update_markers_all()
initial_state = pscene.initialize_state(initial_state.Q, force_fit_binding=True)

In [4]:
GRASP_SAMPLE = 30
STABLE_SAMPLE = 50
SEARCH_SAMPLE_RATIO = 1000
MAX_TIME = 100

In [5]:
VERBOSE = True
VISUALIZE = True
PLAY_RESULT = True
SHOW_STATE = True

## Solve in PDDLStream

In [6]:
########################################################
#################### Solve problem  ####################
obj_pscene = pscene.subject_dict[pscene.subject_name_list[0]]
obj_pscene.geometry.color = (0.8, 0.2, 0.2, 1)
gscene.update_markers_all()
goal_pairs=[(obj_pscene.oname, 'gp')]

gtimer = GlobalTimer.instance()
gtimer.reset(stack=STACK_TIMELOG)


res, plan, log_dict = solve_in_pddlstream(pscene, mplan, ROBOT_NAME, TOOL_NAME, HOME_POSE, goal_pairs,
                                          TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS,
                                          GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO,
                                          use_pybullet_gui=USE_PYBULLET_GUI, USE_MOVEIT_IK=USE_MOVEIT_IK, 
                                          TIMED_COMPLETE=TIMED_COMPLETE,
                                          IK_TRY_NUM=IK_TRY_NUM, IK_TIMEOUT_SINGLE=IK_TIMEOUT_SINGLE, VERBOSE=VERBOSE)

log_dict.update(mplan.result_log)
log_dict.update(GlobalLogger.instance())

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
Objects: {1L: 'obs5', 2L: 'obj_0', 3L: 'gp', 4L: 'obs6', 5L: 'wp_bot', 6L: 'obs4', 7L: 'floor', 8L: 'obs2', 9L: 'obs3', 10L: 'obs0', 11L: 'obs1', 12L: 'base', 13L: 'wp', 14L: 'gp_bot', 15L: 'obj_1', 16L: 'obj_2'}
================ MAKE PROBLEM ======================
IK checkers: ['GraspChecker', 'ReachChecker', 'LatticedChecker']
MP checkers: []
timeout motion : 2
Robot: 0
Movable: [16L, 15L, 2L]
Fixed: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
body 16 - surface 1
body 16 - surface 3
body 16 - surface 4
body 16 - surface 5
body 16 - surface 6
body 16 - surface 7
body 16 - surface 8
body 16 - surface 9
body 16 - surface 10
body 16 - surface 11
body 16 - surface 12
body 16 - surface 13
body 16 - surface 14
body 15 - surface 1
body 15 - surface 3
body 15 - surface 4
body 15 - surface 5
body 15 - surface 6
body 15 - surface 7
body 15 - surface 8
body 15 - surface 9
body 15 - surface 10
body 1

check / plan: True / True
iter=inf, outs=1) plan-free-motion:(q0, q7)->[(c8)]
subject/actor: None / grip0
Motion Plan Failure
check / plan: True / False
iter=inf, outs=0) plan-free-motion:(q8, q0)->[]
iter=1, outs=1) sample-pose:(2, 3)->[(p4)]
Check Feas Fail: GraspChecker
iter=inf, outs=0) inverse-kinematics:(2, p4, g10)->[]
iter=2, outs=1) sample-pose:(2, 3)->[(p5)]
inverse_kinematics fail to approach
inverse_kinematics fail to approach
inverse_kinematics fail to approach
inverse_kinematics fail to approach
inverse_kinematics fail to approach
inverse_kinematics fail to approach
inverse_kinematics fail to approach
inverse_kinematics fail to approach
iter=inf, outs=0) inverse-kinematics:(2, p5, g10)->[]
iter=3, outs=1) sample-pose:(2, 3)->[(p6)]
Check Feas Fail: GraspChecker
iter=inf, outs=0) inverse-kinematics:(2, p6, g10)->[]
iter=4, outs=1) sample-pose:(2, 3)->[(p7)]
Check Feas Fail: LatticedChecker
iter=inf, outs=0) inverse-kinematics:(2, p7, g10)->[]
iter=5, outs=1) sample-pose:(2

check / plan: True / True
iter=inf, outs=1) plan-free-motion:(q0, q9)->[(c11)]
subject/actor: None / grip0
check / plan: True / True
iter=inf, outs=1) plan-free-motion:(q10, q0)->[(c12)]
subject/actor: obj_0 / gp
check / plan: True / True
iter=inf, outs=1) plan-holding-motion:(q9, q10, 2, g19)->[(c13)]
Summary: {complexity: 2, cost: 0.000, evaluations: 182, iterations: 3, length: 2, run_time: 71.177, sample_time: 70.937, search_time: 0.239, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 431558 | p_success: 0.982 | overhead: 0.000
External: sample-grasp | n: 395938 | p_success: 0.991 | overhead: 0.000
External: inverse-kinematics | n: 4785170 | p_success: 0.139 | overhead: 0.028
External: plan-free-motion | n: 197815 | p_success: 0.566 | overhead: 0.817
External: plan-holding-motion | n: 46530 | p_success: 0.933 | overhead: 0.386
External: test-cfree-pose-pose | n: 84299 | p_success: 1.000 | overhead: 0.000
External: test-

In [7]:
if VISUALIZE and PLAY_RESULT and res:
    play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
                   body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)


In [8]:
print({k:v for k,v in log_dict['body_names'].items() if "obj" in v})
print({k:v for k,v in log_dict['body_names'].items() if v.endswith('p')})

{16L: 'obj_2', 2L: 'obj_0', 15L: 'obj_1'}
{3L: 'gp', 13L: 'wp'}


In [9]:
for k, v in pscene.actor_dict.items():
    print("{}:{}-{}".format(k, v.name, v.geometry.name))

grip0:grip0-grip0
gp:gp-gp
wp:wp-wp


## Old version

In [8]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


In [9]:
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [10]:
from_state = initial_state.copy(pscene)
from_state.Q = HOME_POSE
goal_nodes = [('gp','wp','wp')]
ppline.search(from_state, goal_nodes, verbose=False, display=False, 
              timeout_loop=100, multiprocess=True, timeout=1, add_homing=True)

Use 20/20 agents
Goal reached
++ adding return motion to acquired answer ++
Goal reached
======================= terminated 9: first answer acquired ===============================
======================= terminated 6: first answer acquired from other agent ===============================
======================= terminated 4: first answer acquired from other agent ===============================
======================= terminated 1: first answer acquired from other agent ===============================
======================= terminated 11: first answer acquired from other agent ===============================
======================= terminated 10: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired from other agent ===============================
======================= terminated 16: first answer acquired from other agent ===============================
======================= terminated 8: first answer a

In [11]:
snode_schedule = ppline.tplan.get_best_schedule()

In [12]:
ppline.play_schedule(snode_schedule)

('wp', 'wp', 'wp')->('wp', 'grip0', 'wp')
('wp', 'grip0', 'wp')->('wp', 'gp', 'wp')
('wp', 'gp', 'wp')->('wp', 'gp', 'grip0')
('wp', 'gp', 'grip0')->('wp', 'gp', 'wp')
('wp', 'gp', 'wp')->('grip0', 'gp', 'wp')
('grip0', 'gp', 'wp')->('wp', 'gp', 'wp')
('wp', 'gp', 'wp')->('grip0', 'gp', 'wp')
('grip0', 'gp', 'wp')->('gp', 'gp', 'wp')
('gp', 'gp', 'wp')->('gp', 'gp', 'grip0')
('gp', 'gp', 'grip0')->('gp', 'gp', 'wp')
('gp', 'gp', 'wp')->('gp', 'grip0', 'wp')
('gp', 'grip0', 'wp')->('gp', 'wp', 'wp')
('gp', 'wp', 'wp')->('gp', 'wp', 'wp')
